<a href="https://colab.research.google.com/github/apadkavyrava/Author-detection/blob/master/Autor_name_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Cated sourse file with terminal and re-named to article_100.csv
```
# ANNAs-MacBook:Downloads ANNA$ cat articles.csv | head -n 100 > articles_100.csv
```





Import and look into data

In [0]:
import pandas as pd
import spacy
nlp = spacy.load('en_core_web_sm')

raw_data = pd.read_csv('articles_100.csv')

#have a look how file looks like
raw_data.head()
raw_data.columns
#we need only authors
data = raw_data['author'].astype(str)
print(data.head(20))



0     http://www.dailymail.co.uk/home/search.html?s=...
1     http://www.dailymail.co.uk/home/search.html?s=...
2                                          Josh Jackman
3     http://www.dailymail.co.uk/home/search.html?s=...
4                                            Ben Rumsby
5                                          Zahra Mulroy
6                                         James Titcomb
7     http://www.dailymail.co.uk/home/search.html?s=...
8                                           James Caven
9                              Ben Chapman, Ben Chapman
10                                             BBC News
11    http://www.dailymail.co.uk/home/search.html?s=...
12    http://www.dailymail.co.uk/home/search.html?s=...
13                                        Matthew Field
14    http://www.dailymail.co.uk/home/search.html?s=...
15                                        Jessica Glass
16    http://www.dailymail.co.uk/home/search.html?s=...
17                                           Ned

Authors contains in single words. So we can use part of speech tagging to extract authors. Suppose it will be the pairs of proper nouns.

In [0]:


from spacy.matcher import Matcher
matcher = Matcher(nlp.vocab)

def extract_full_name(int_doc):
  pattern = [{'POS': "PROPN"}, {'POS': "PROPN"}]
  matcher.add('FULL_NAME', None, pattern)
  matches = matcher(int_doc)
  for match_id, start, end in matches:
    span = int_doc[start:end]
    return span.text

tokens = data.apply(lambda x: nlp(str(x)))
names = tokens.apply(lambda x: extract_full_name(x))
print(names)

0       Associated Press
1       Associated Press
2           Josh Jackman
3     Cheyenne Macdonald
4             Ben Rumsby
             ...        
93       Aatif Sulleyman
94      Associated Press
95       Julia Pritchard
96           Adam Lusher
97           Jack Austin
Name: author, Length: 98, dtype: object


Pairs Proper Nouns (PROPN) includs organisations. There is no chance to apply filter, because we have no marks about Proper name.


Use named entity recognition and print labes.

In [0]:
import spacy

train = data[11:22]
nlp = spacy.load("en_core_web_sm")
records = []
for raw in data:
  authors =[]
  doc = nlp(raw)
  for ent in doc.ents:
    authors.append((ent.text, ent.label_))
  records.append(authors)
print(records)



[[('Associated Press', 'ORG')], [('Associated Press', 'ORG')], [('Josh Jackman', 'PERSON')], [('Reuters', 'ORG'), ('Cheyenne Macdonald', 'PERSON')], [('Ben Rumsby', 'PERSON')], [('Zahra Mulroy', 'PERSON')], [('James Titcomb', 'PERSON')], [('Heidi Parker', 'PERSON'), ('Dailymail.com', 'ORG')], [('James Caven', 'PERSON')], [('Ben Chapman', 'PERSON'), ('Ben Chapman', 'PERSON')], [('BBC News', 'ORG')], [('Dailymail.com', 'ORG')], [('Matthew Wright', 'PERSON'), ('Dailymail.com', 'ORG')], [('Matthew Field', 'PERSON')], [], [('Jessica Glass', 'PERSON')], [('Marc Jackson', 'PERSON'), ('MailOnline', 'ORG')], [('Ned Simons', 'PERSON')], [('BBC News', 'ORG')], [('Associated Press', 'ORG')], [('Dailymail.com', 'ORG')], [('Tony Ward', 'PERSON')], [('Tony Ward', 'PERSON')], [('Jane Foster', 'PERSON')], [('Sophie Campbell', 'PERSON')], [('Caroline Mortimer', 'PERSON'), ('Caroline Mortimer', 'PERSON')], [('Shivali Best For Mailonline', 'ORG')], [('Joseph Patrick McCormick', 'PERSON')], [('By Press Ass

We extracted proper noun and labels from the rows. Now apply filter to extract only authors and replace organisations, unknows etc. with None value

In [0]:
new_data = []
for record in records:
  pers_rec = [t for t in record if t[1] == 'PERSON']
  if len(pers_rec) ==0:
    new_data.append(None)
  else: 
    new_data.append(pers_rec[0][0])
print(new_data)

[None, None, 'Josh Jackman', 'Cheyenne Macdonald', 'Ben Rumsby', 'Zahra Mulroy', 'James Titcomb', 'Heidi Parker', 'James Caven', 'Ben Chapman', None, None, 'Matthew Wright', 'Matthew Field', None, 'Jessica Glass', 'Marc Jackson', 'Ned Simons', None, None, None, 'Tony Ward', 'Tony Ward', 'Jane Foster', 'Sophie Campbell', 'Caroline Mortimer', None, 'Joseph Patrick McCormick', None, 'Andrew Gilpin', 'Chris Stocks', None, 'Snejana Farberov', 'Sara C Nelson', 'Emily Crane', 'Jack Torrance', None, None, 'Thomas Burrows', None, None, None, None, None, None, 'Jack Wilson', 'Kate Forrester', None, 'Alex Matthews-King', None, 'Aoibhinn Mcbride', 'James Whaling', 'Elizabeth Aubrey', 'Aaron Flanagan', 'Kelly Mclaughlin', 'Luke Brown', 'Rod Ardehali', 'Olivia Buxton Smith', None, None, 'Sophie Curtis', 'Amy Packham', 'Tim Wallace', 'Tom Hoggins', 'Molly Holt', None, None, None, 'Jeff Farrell', None, 'Lucy Mapstone', 'Lucy Mapstone', 'Charlotte Neal', 'Catherine Wyatt', 'Ryan Smith', None, None, Non

add colums Authors_clean in source file with new filtered data.

In [0]:
article = raw_data.copy()
article['authors_clean']=new_data
article.head()

,Unnamed: 0,author,content,description,publishedAt,source,title,url,urlToImage,authors_clean
0,0,http://www.dailymail.co.uk/home/search.html?s=...,"KAMPALA, Uganda (AP) - Rights groups on Monday...","KAMPALA, Uganda (AP) - Rights groups on Monday...",2017-11-13T16:35:28Z,"{'id': None, 'name': 'Dailymail.co.uk'}","Sudan's president, wanted by the ICC, visits U...",http://www.dailymail.co.uk/wires/ap/article-50...,http://i.dailymail.co.uk/i/pix/m_logo_636x382p...,None
1,1,http://www.dailymail.co.uk/home/search.html?s=...,MIAMI (AP) - The U.S. Supreme Court agreed Mon...,MIAMI (AP) - The U.S. Supreme Court agreed Mon...,2017-11-13T16:35:12Z,"{'id': None, 'name': 'Dailymail.co.uk'}",Florida man back at Supreme Court with 1st Ame...,http://www.dailymail.co.uk/wires/ap/article-50...,http://i.dailymail.co.uk/i/pix/m_logo_636x382p...,None
2,2,Josh Jackman,The Spice Girls are set to reunite for a TV sp...,Viva Forever!,2017-11-13T16:34:58Z,"{'id': None, 'name': 'Pinknews.co.uk'}",The Spice Girls are reuniting,http://www.pinknews.co.uk/2017/11/13/the-spice...,http://www.pinknews.co.uk/images/2017/11/Getty...,Josh Jackman
3,3,http://www.dailymail.co.uk/home/search.html?s=...,Ride-hailing firm Lyft Inc said on Monday it w...,Ride-hailing firm Lyft Inc said on Monday it w...,2017-11-13T16:34:58Z,"{'id': None, 'name': 'Dailymail.co.uk'}",Lyft to drive into Canada in first internation...,http://www.dailymail.co.uk/sciencetech/article...,http://i.dailymail.co.uk/i/pix/2017/11/13/16/4...,Cheyenne Macdonald
4,4,Ben Rumsby,"T im Gudgin, the long-time voice of BBC televi...","Tim Gudgin, the long-time voice of BBC televis...",2017-11-13T16:34:45Z,"{'id': None, 'name': 'Telegraph.co.uk'}","Tim Gudgin, voice of Saturday tea-time footbal...",http://www.telegraph.co.uk/football/2017/11/13...,http://www.telegraph.co.uk/content/dam/footbal...,Ben Rumsby
